In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.hash import hash_func_builder
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic
from JAxtar.hash import HashTable

/tmp/ipykernel_24708/2078866209.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
count = int(1e3)
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(2),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.make_lookup_table(puzzle.State, 1, int(1e5))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)

0.20320701599121094


In [4]:
batch = 4000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
for i in range(18):
    sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(i + 256),count))
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    batched_sample = jax.tree_util.tree_map(lambda x, y: jnp.concatenate([x, y]), sample, jax.vmap(puzzle.State.default)(jnp.arange(batch - count)))
    filled = jnp.concatenate([jnp.ones(count), jnp.zeros(batch - count)], dtype=jnp.bool_)
    start = time.time()
    table, inserted = parallel_insert(table, batched_sample, filled)
    print(time.time()-start)
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    print(jnp.mean(found), jnp.mean(inserted))

0.6140413284301758
1.0 0.25
0.001148223876953125
1.0 0.25
0.0011126995086669922
1.0 0.25
0.0008466243743896484
1.0 0.25
0.0007474422454833984
1.0 0.25
0.0008151531219482422
1.0 0.25
0.00091552734375
1.0 0.25
0.0010848045349121094
1.0 0.25
0.0008347034454345703
1.0 0.25
0.0007369518280029297
1.0 0.25
0.0007338523864746094
1.0 0.25
0.0009334087371826172
1.0 0.25
0.000957489013671875
1.0 0.25
0.000995635986328125
1.0 0.25
0.0009443759918212891
1.0 0.25
0.0007560253143310547
1.0 0.25
0.0009715557098388672
1.0 0.25
0.00084686279296875
1.0 0.25


In [5]:
count = int(1e6)
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(2),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.make_lookup_table(puzzle.State, 1, int(1e7))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)
#print(seeds, idx, table_idx, found)

0.22794723510742188


In [6]:
batch = 4000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
for i in range(18):
    inserteds = []
    sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(i + 256),count))
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    same_ratio = jnp.mean(found)
    for j in trange(0, count, batch):
        table, inserted = parallel_insert(table, sample[j:j+batch], jnp.ones(batch, dtype=jnp.bool_))
        inserteds.append(inserted)
    inserteds = jnp.concatenate(inserteds)
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    print(jnp.mean(found), same_ratio, jnp.mean(inserteds))

100%|██████████| 250/250 [00:01<00:00, 194.14it/s]


1.0 0.0 1.0


100%|██████████| 250/250 [00:00<00:00, 374.04it/s]


1.0 0.0 1.0


100%|██████████| 250/250 [00:00<00:00, 373.96it/s]


...

1.0 0.0 1.0


100%|██████████| 250/250 [00:01<00:00, 137.46it/s]


1.0 2e-06 0.999998


100%|██████████| 250/250 [00:02<00:00, 95.45it/s] 


1.0 0.0 1.0


In [7]:
another_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(123),count))
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print(jnp.sum(found) / count)

1e-06


In [8]:
start = time.time()
inserteds = []
for i in trange(count // batch):
    table, inserted = parallel_insert(table, another_sample[i*batch:(i+1)*batch], jnp.ones(batch, dtype=jnp.bool_))
    inserteds.append(inserted)
print(jnp.sum(jnp.concatenate(inserteds)) / count)
print("insert time:", time.time()-start)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, new_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)

100%|██████████| 250/250 [00:05<00:00, 42.89it/s]


0.999999
insert time: 5.835551023483276
check time: 0.15099239349365234
1.0
check time: 0.2289562225341797
1e-06
check time: 0.5082619190216064
1.0
